In [413]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import MeCab
import unicodedata
import re
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
import os
import io

tf.enable_eager_execution()

In [414]:
# load txt file
def load_def(path):
    # open a txt file as read only
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    
    return lines

In [415]:
# create each languages list
def create_lang_list(num_example):
    # load txt file
    lines = load_def("dataset/raw.txt")

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_example]]

    return zip(*word_pairs)

# translate English to Japanese

In [416]:
# convert unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFC', s)
                    if unicodedata.category(c) != 'Mn')

# preprocess
def preprocess_sentence(w):
    # check japanese lang
    p = re.compile('[\u3041-\u309F\u30A1-\u30FF\uFF66-\uFF9F\u4E00-\u9FD0\u309B\u3099\uFF9E\u309C\u309A\uFF9F]+')
    if p.search(w):
        # Morphological analysis for japanese lang
        m = MeCab.Tagger("-Owakati")
        w = m.parse(w)
    w = unicode_to_ascii(w.lower().strip())
    # create a space between word and the punctuation
    w = re.sub(r"([?!¿.,。])", r" \1 ", w)
    # replacing everything with space except(a-z, A-Z, ".",  "?",  "!",  ",", "-", "ー", , "。", "Kanji", "Katakana", "Hiragana")
    w = re.sub(r"[^a-zA-Z\u3041-\u309F\u30A1-\u30FF\uFF66-\uFF9F\u4E00-\u9FD0\u309B\u3099\uFF9E\u309C\u309A\uFF9F?.!,。¿\-/ {1,}/]+",  " ", w)
    w = w.rstrip().strip()
    
    # add a start and end  token to the sentence
    # model know when to start and end
    w = "<start> " + w + " <end>"
    return w

In [417]:
# check word
en_sentence =u"May I borrow this book?"
ja_sentence = u"プールに行きたい。でも今日は雨."
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(ja_sentence))

<start> may i borrow this book ? <end>
<start> プール に 行き たい  。  でも 今日 は 雨  . <end>


In [418]:
en, ja = create_lang_list(1000)
print(len(en))
print(ja[:10])

1000
('<start> あなた は 戻っ た の ね ハロルド  ? <end>', '<start> 俺 の 相手 は シャーク だ  。 <end>', '<start> 引き換え だ ある 事 と ある 物 の <end>', '<start> もう いい よ ごちそうさま ううん <end>', '<start> もう 会社 に は 来 ない で くれ 電話 も する な <end>', '<start> きれい だ  。 <end>', '<start> 連れ て 行け 殺し そう だ わかっ た か  ? <end>', '<start> 殺し た の か  ! <end>', '<start> わぁ   !  いつも すみません  。  いい の よ   。 <end>', '<start> カンパニー の 元 社員 が <end>')


In [419]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [420]:
def tokenize(lang):
    # vectorize a text corpus
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters=' ')
    # updates internal vocabulary based on a list of texts
    # e.g. "[this place is good ]"→{this:1, place:2, is:3, good:4} "
    lang_tokenizer.fit_on_texts(lang)
    # Transforms each text in texts to a sequence of integers.
    tensor = lang_tokenizer.texts_to_sequences(lang)
    # transform a list of num sample into a 2D Numpy array of shape 
    # Fixed length because length of sequence of integers are different
    # return (len(sequences), maxlen)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                          padding='post')
    return tensor, lang_tokenizer

In [421]:
# example
tokenize(['this place is good', "hello world", "today is so cold"])

(array([[2, 3, 1, 4],
        [5, 6, 0, 0],
        [7, 1, 8, 9]], dtype=int32),
 <keras_preprocessing.text.Tokenizer at 0x1a4b455cf8>)

In [422]:
# create a clean input, output pairs
def load_dataset(num_example):
    input_lang, target_lang= create_lang_list(num_example)
    input_tensor, input_lang_tokenize = tokenize(input_lang)
    print(input_lang_tokenize.word_index["hello"])
    target_tensor, target_lang_tokenize = tokenize(target_lang)

    return input_tensor, target_tensor, input_lang_tokenize, target_lang_tokenize

In [423]:
# limit datasize for test
num_example = 1000
# get data
input_tensor, target_tensor, input_lang, target_lang = load_dataset(num_example)
# Calculate max_length of the target tensors
max_length_target, max_length_input = max_length(target_tensor), max_length(input_tensor)

398


In [424]:
# create trainnig set and validation set
input_tensor_train, input_tensor_val, \
    target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2, random_state=42)

# show length
print(len(input_tensor_train), len(input_tensor_val), len(target_tensor_train), len(target_tensor_val))

800 200 800 200


In [425]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            # Index number assigned to each word
            print("%d----->%s" % (t, lang.index_word[t]))

In [426]:
print("input lang: index to word mapping")
convert(input_lang, input_tensor_train[0])
print("output lang: index to word mapping")
convert(target_lang, target_tensor_train[0])

input lang: index to word mapping
1-----><start>
605----->asamoto
184----->won
19----->t
273----->last
38----->like
22----->this
16----->!
2-----><end>
output lang: index to word mapping
1-----><start>
676----->asamoto
4----->は
42----->この
677----->まま
52----->じゃ
198----->いけ
16----->ない
17----->と
136----->思う
11----->。
2-----><end>


In [434]:
# create a dataset
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)
embedding_dim = 100
units = 1024
vocab_inp_size = len(input_lang.word_index) + 1
print('Total unique words in the input: %s' % len(input_lang.word_index))
vocab_tar_size = len(target_lang.word_index) + 1
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
print(dataset)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

Total unique words in the input: 1817
<DatasetV1Adapter shapes: ((47,), (32,)), types: (tf.int32, tf.int32)>
<DatasetV1Adapter shapes: ((64, 47), (64, 32)), types: (tf.int32, tf.int32)>


In [428]:
example_input_batch, example_target_batch =  next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([Dimension(64), Dimension(47)]),
 TensorShape([Dimension(64), Dimension(32)]))

# encoder and decoder model

In [431]:
# load glove
def load_glove():
    embeddings_dictionary = {}
    glove_file = open("dataset/glove.6B/glove.6B.100d.txt", encoding="utf-8")
    
    for line in glove_file:
        records =  line.split()
        word = records[0]
        vector_dimensions = asarray(records[1:], dtype="float32")
        embeddings_dictionary[word] =  vector_dimensions
    glove_file.close()
    
    num_words = min(num_example, vocab_inp_size)
    embedding_matrix = zeros((num_words, embedding_dim))
    for word, index in input_lang.word_index.items():
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    return embeddings_dictionary, embedding_matrix

In [ ]:
embeddings_dictionary, embedding_matrix = load_glove()

In [433]:
print(embeddings_dictionary["hello"])
print(embedding_matrix[398])

[ 0.26688    0.39632    0.6169    -0.77451   -0.1039     0.26697
  0.2788     0.30992    0.0054685 -0.085256   0.73602   -0.098432
  0.5479    -0.030305   0.33479    0.14094   -0.0070003  0.32569
  0.22902    0.46557   -0.19531    0.37491   -0.7139    -0.51775
  0.77039    1.0881    -0.66011   -0.16234    0.9119     0.21046
  0.047494   1.0019     1.1133     0.70094   -0.08696    0.47571
  0.1636    -0.44469    0.4469    -0.93817    0.013101   0.085964
 -0.67456    0.49662   -0.037827  -0.11038   -0.28612    0.074606
 -0.31527   -0.093774  -0.57069    0.66865    0.45307   -0.34154
 -0.7166    -0.75273    0.075212   0.57903   -0.1191    -0.11379
 -0.10026    0.71341   -1.1574    -0.74026    0.40452    0.18023
  0.21449    0.37638    0.11239   -0.53639   -0.025092   0.31886
 -0.25013   -0.63283   -0.011843   1.377      0.86013    0.20476
 -0.36815   -0.68874    0.53512   -0.46556    0.27389    0.4118
 -0.854     -0.046288   0.11304   -0.27326    0.15636   -0.20334
  0.53586    0.59784   

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding_dim = embedding_dim
        